## Introduction 

The aim of this lab experience is to explore **clustering** in a computing environment. 


We will use the `Iris` flower dataset once again. The `Iris` flower fataset is a classic dataset used to identify types of flowers based on features describing their petals. We will load the dataset directly from the ```scikit-learn``` machine learning library.


After loading the dataset, we will extract two of the attributes and will run the K-means algorithm to obtain 3 clusters. 

### 1. The dataset
 

0.   Let's load the `Iris` dataset and import the NumPy and Matplotlib libraries. We will also print the description of the iris dataset.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

# import the iris dataset
iris = datasets.load_iris()
print(iris.DESCR)

According to the desription, the dataset consists of samples described by four predictor attributes (`sepal length`, `sepal width`, `petal length`, `petal width`, in centimeters) plus a label which can take on three discrete values (`Iris-Setosa`, `Iris-Versicolour` and `Iris-Virginica`). 

We will only work with the first two attributes (`sepal length`, `sepal width`), which will be stored in the variable `X`. In this lab we do not need the label, as we are not classifying them.

In [2]:
X = iris.data[:, :2]  # we will consider the first two features.


In the following code we plot the samples in the 2D space where they live, as they are described by two attributes. 

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
ax.set_aspect('equal')

ax.scatter(X[:, 0], X[:, 1], edgecolor='black', c = 'w')

ax.set_xlabel(iris.feature_names[0], fontsize=14)
ax.set_ylabel(iris.feature_names[1], fontsize=14)
ax.grid(alpha=0.3)
ax.set_xlim(X[:, 0].min() - 0.5, X[:, 0].max() + 0.5)
ax.set_ylim(X[:, 1].min() - 0.5, X[:, 1].max() + 0.5)
plt.show()

### 2. K-means initialisation
 

K-means uses K prototypes to represent each cluster and clusters are formed by assigning samples to the closest prototype. In this lab activity, we will use cluster our data into 3 clusters, i.e. K=3.

The first step in K-means is then to create the K prototypes. The following code creates 3 prototypes for K-means in 3 random locations.

In [4]:
my_seed = 6 #the seed is used by numpy's random method to generate random numbers
np.random.seed(seed = my_seed)

k = 3 # set the k value of k-means
centers3 = np.random.normal(size=[k, 2]) + np.ones((k,1)) * np.mean(X, axis=0)

Then, the samples in the dataset will be assigned to the nearest prototype, which will result in K=3 clusters. In the following cell, we create the first clustering arrangement and plot it.

In [ ]:
from scipy.spatial import distance

# Find the euclidean distance between every point and every cluster.
distanceMatrix3 = distance.cdist(X, centers3, 'euclidean')

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)
ax.set_aspect('equal')

overalDistToClusters3 = 0.0

# Make the labe of each point be the closest cluster.
whichCenterNearest = np.argsort(distanceMatrix3, axis=1)[:, 0]

for index in range(len(X)):  
  overalDistToClusters3 += distanceMatrix3[index][whichCenterNearest[index]]
  if whichCenterNearest[index] == 0:
    ax.scatter(X[index, 0], X[index, 1], edgecolor='red', c = 'w')
  if whichCenterNearest[index] == 1:
    ax.scatter(X[index, 0], X[index, 1], edgecolor='blue', c = 'w')
  if whichCenterNearest[index] == 2:
    ax.scatter(X[index, 0], X[index, 1], edgecolor='green', c = 'w')
    
# Plot the centers that we calculated for k-means in the previous cell
ax.scatter(centers3[0, 0], centers3[0, 1], color='red', marker = 'X', s= 100)
ax.scatter(centers3[1, 0], centers3[1, 1], color='blue', marker = 'X', s = 100)
ax.scatter(centers3[2, 0], centers3[2, 1], color='green', marker = 'X', s = 100)

ax.set_xlabel(iris.feature_names[0], fontsize=14)
ax.set_ylabel(iris.feature_names[1], fontsize=14)
ax.grid(alpha=0.3)
plt.show()

As you can see, there are 3 prototypes represented by a cross of different colours. Samples are represented by a circle whose colour corresponds to the cluster they have been assigned to. The 3 resulting clusters consist of samples that are close to one another. However, samples in different clusters are also close to one another.  

### 3. K-means iterative update
 

K-means proceeds iteratively by


1.   Updating the prototypes as the centre of each cluster
2.   Reassigning the samples to the new prototypes

Until the stop criterion it met.

The following code runs K-means and stops when the number of iterations is larger than the value  `max_iteration` or the difference between the total distances of the consecutive iterations is smaller than the value `tol`. The clustering arrangement in each iteration will also be plotted.

In [ ]:

tol = 0.000001
max_iteration = 100
difference = np.inf
iteration = 0
overalDistToClusters3 = [np.inf]

edgecolorlist = ['red', 'blue', 'green']

while difference>tol and iteration<max_iteration:
    
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.set_xlabel(iris.feature_names[0], fontsize=14)
    ax.set_ylabel(iris.feature_names[1], fontsize=14)
    ax.grid(alpha=0.3)
    ax.set_xlim(4, 8)
    ax.set_ylim(1, 4.5)
    
    overalDistToClusters3_new = 0.0
    distanceMatrix3 = distance.cdist(X, centers3, 'euclidean')
    whichCenterNearest = np.argsort(distanceMatrix3, axis=1)[:, 0]

    for index in range(len(X)):
        overalDistToClusters3_new += distanceMatrix3[index][whichCenterNearest[index]]
    difference = overalDistToClusters3[-1] - overalDistToClusters3_new
    overalDistToClusters3.append(overalDistToClusters3_new)
    
    for i in range(k):
        indx = whichCenterNearest==i
        if indx.any():
            centers3[i,:] = np.mean(X[indx,:], axis = 0)
    
        ax.scatter(X[indx, 0], X[indx, 1], edgecolor=edgecolorlist[i], c = 'w')
        ax.scatter(centers3[i, 0], centers3[i, 1], color=edgecolorlist[i], marker = 'X', s= 100, alpha=0.9) 

    iteration +=1
    ax.set_title('iteration = {:d}, difference = {:.4f}'.format(iteration, difference))


plt.show()

How many times has K-means updated the clustering solution?

Use the notions of intra-cluster sample scatter and inter-cluster sample scatter to visually compare the first (random) clustering arrangement and the final one. Why do we consider the final clustering arrangement to be better than the first one?

The following code plots the overall distance from each sample to the prototype they have been assigned to in each iteration. You will remember that this distance is related to the intra-cluster sample scatter.

In [ ]:

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)

ax.set_xlabel("Iteration", fontsize=14)
ax.set_ylabel("Overall distance", fontsize=14)
ax.grid(alpha=0.3)

ax.plot(range(1,len(overalDistToClusters3)), overalDistToClusters3[1:], '--', marker = 'o')
ax.set_ylim([0, overalDistToClusters3[1]*1.05])

plt.show()

Based on this plot, would you say that the quality of the clustering arrangement is improving as K-means progresses?

### 4. Local solutions in K-means
 

The K-means algorithm looks for the clustering arrangement that minimises the intra-cluster sample scatter. However, there is no guarantee that K-means will find the so-called global best, but a local one.

We could expect K-means to return a different clustering arrangement if the initial location of the prototypes changed. In the previous cells, by changing the value of `my_seed` the initialisation of the prototypes change. Have a go at re-running K-means with different prototype initialisations. In order to do so, simply change the value of `my_seed` and re-run the code below. You can use integer numbers, for instance from 2 to 10. What do you observe? Do you always obtain the same solution?

### 5. Conclusions
 
In this lab experience we have explored clustering. We have applied the K-means algorithm to a projection of the `Iris` dataset on a 2D space, with K=3. First, we have initialised K-means by randomly generating 3 prototypes and creating an initial clustering arrangement. Then, we have iteratively updated the prototypes and reassigned the samples to the nearest prototype. Finally, we have observed that changing the initial locations of the prototypes can change the final solution.

 